# Try reading gnomAD data into memory

In [4]:
using Revise
using gnomAD

┌ Info: Precompiling gnomAD [a5d00db5-5f0a-49b9-aaff-6f4121a13ee1]
└ @ Base loading.jl:1423


In [5]:
data = "/Users/biona001/.julia/dev/gnomAD/data/gnomad.genomes.r2.1.1.nfe.common.adj.ld.bm"
bm = hail_block_matrix(data);

In [16]:
bm[1, 1]

0.9999999999999998

In [6]:
@time begin
    out = zeros(10, 10)
    for i in 1:10, j in 1:10
        out[i, j] = bm[i, j]
    end
end
out

2023-03-01 16:39:40.545 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
 21.392457 seconds (11.79 k allocations: 673.166 KiB, 0.09% compilation time)


Initializing Hail with default parameters...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://dn52ecqn.sunet:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /Users/biona001/.julia/dev/gnomAD/test/hail-20230301-1639-0.2.109-b71b065e4bb6.log


10×10 Matrix{Float64}:
 1.0  0.0326361  -0.0181891   0.0166341   …   0.0179389   -0.00994853
 0.0  1.0        -0.00041394  0.00540477      0.00211069   0.00171555
 0.0  0.0         1.0         0.0177243       0.0         -0.000284646
 0.0  0.0         0.0         1.0             0.00627614   0.00131174
 0.0  0.0         0.0         0.0            -0.0045831    0.00119736
 0.0  0.0         0.0         0.0         …   0.0145138    0.00530853
 0.0  0.0         0.0         0.0             0.00089687   0.00328037
 0.0  0.0         0.0         0.0             0.123663     0.377178
 0.0  0.0         0.0         0.0             1.0          0.00725709
 0.0  0.0         0.0         0.0             0.0          1.0

In [18]:
@time bm[1:1000, 1:1000]

  0.471002 seconds (67 allocations: 7.632 MiB)


1000×1000 Matrix{Float64}:
 1.0  0.0326361  -0.0181891   0.0166341   …   0.00767459    0.0104462
 0.0  1.0        -0.00041394  0.00540477     -0.10473      -0.00184638
 0.0  0.0         1.0         0.0177243      -0.000167982  -0.00929269
 0.0  0.0         0.0         1.0            -2.1684e-19   -0.00329412
 0.0  0.0         0.0         0.0             1.2618e-5    -0.000859109
 0.0  0.0         0.0         0.0         …   2.98156e-19   0.0507852
 0.0  0.0         0.0         0.0             2.88077e-5   -0.0117684
 0.0  0.0         0.0         0.0            -0.000116338  -0.0104104
 0.0  0.0         0.0         0.0             2.67669e-5   -0.00728978
 0.0  0.0         0.0         0.0             2.331e-5     -0.00253932
 0.0  0.0         0.0         0.0         …   0.00150567   -0.00353229
 0.0  0.0         0.0         0.0             0.00509611    0.0109818
 0.0  0.0         0.0         0.0            -0.0126056     0.0126193
 ⋮                                        ⋱            

In [19]:
# indexing non-contiguous region is slow
@time bm[1:3, 1:2:100]

 22.868453 seconds (61.51 k allocations: 3.108 MiB, 0.31% compilation time)


3×50 Matrix{Float64}:
 1.0  -0.0181891   0.00802797  -0.0421512  …  0.0143534     0.0172358
 0.0  -0.00041394  0.00232164  -0.0112066     5.39059e-5   -0.0188801
 0.0   1.0         0.0         -0.0282011     0.000733417  -0.0277042

In [44]:
using PyCall
py"""
from hail.linalg import BlockMatrix
import numpy as np
def convert(bm, range_start, range_end):
    return bm[range_start:range_end, range_start:range_end].to_numpy()
"""
@time py"convert"(bm.bm, 0, 100)

  0.827913 seconds (33.87 k allocations: 1.972 MiB, 1.93% compilation time)


100×100 Matrix{Float64}:
 1.0  0.0326361  -0.0181891   0.0166341   …   0.0172358    -0.00857776
 0.0  1.0        -0.00041394  0.00540477     -0.0188801     0.05925
 0.0  0.0         1.0         0.0177243      -0.0277042    -0.000268512
 0.0  0.0         0.0         1.0            -0.000468342  -0.000966708
 0.0  0.0         0.0         0.0            -7.47553e-5   -0.00298802
 0.0  0.0         0.0         0.0         …   0.00207735    0.00152017
 0.0  0.0         0.0         0.0             0.00256425    0.00136302
 0.0  0.0         0.0         0.0             0.0137416    -0.000680087
 0.0  0.0         0.0         0.0            -0.020671      0.000342287
 0.0  0.0         0.0         0.0             0.00195264   -0.00451867
 0.0  0.0         0.0         0.0         …   0.0018285    -0.00406988
 0.0  0.0         0.0         0.0             0.0152337     0.0100539
 0.0  0.0         0.0         0.0             0.0193       -0.0136828
 ⋮                                        ⋱          

### Python code

In [ ]:
from hail.linalg import BlockMatrix
import numpy as np
bm_files = "gnomad.genomes.r2.1.1.nfe.common.adj.ld.bm"
bm = BlockMatrix.read(bm_files)

# get a slice fast
x = bm[0:100, 0:100]
X = x.to_numpy()
X

In [ ]:
def convert(bm, range1, range2)
    return bm[range1, range2].to_numpy()